In [ ]:
import tensorflow as tf
import random
import matplotlib.pyplot as plt



In [ ]:
print(tf.__version__)

In [ ]:
model = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(96, 96, 3)
)

model.summary()

In [ ]:
def getSubmodel(layer):
    return tf.keras.Model(
        model.input,
        model.get_layer(layer).output
    )

In [ ]:
def createRandomImage(x=100, y=100):
    return tf.random.uniform((x, y, 3), minval=-0.5, maxval=0.5)

In [ ]:
def pltImage(image, title='image'):
    image = image - tf.math.reduce_min(image)
    image = image / tf.math.reduce_max(image)
    plt.imshow(image)
    plt.show()


In [ ]:
image = createRandomImage(96, 96)
pltImage(image)

In [ ]:
def visualizeFilter(layer, it = 50, image = None, index = None):
    submodel = getSubmodel(layer)
    num = submodel.output.shape[-1]
    
    if index is None:
        index = random.randint(0, num - 1)
    assert num > index , 'index too large'
    
    if image is None:
        image = createRandomImage(96, 96)
    
    for i in range(0, it):
        with tf.GradientTape() as tape:
            tape.watch(image)
            out = submodel(tf.expand_dims(image, axis = 0))[:,:,:,index]
            loss = tf.math.reduce_mean(out)
        grads = tape.gradient(loss, image)
        grads = tf.math.l2_normalize(grads)
        image += grads * 10
        
        if(i % 10 == 0):
            print(str(i) + ": loss = ", loss)
        
    pltImage(image, layer)


In [ ]:
layer = 'block1_conv1' #@param ['block1_conv1', 'block1_conv2', 'block2_conv1', 'block2_conv2', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block5_conv1', 'block5_conv2', 'block5_conv3']

In [ ]:

def loadImage(image_path, target_size=(96, 96)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0, 1] range
    return img_array


In [ ]:
image_path = 'test.png'
image = loadImage(image_path)

visualizeFilter(layer, 100, image)